Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

NAME = "Natalie Buchbinder"
COLLABORATORS = ""


---

# Scikit-Learn

This lab will test your knowledge of scikit-learn and your ability to perform basic tasks using this foundational library.

You are welcome (and encouraged) to make use of the scikit-learn documention

https://scikit-learn.org/stable/modules/classes.html

# Problem 1

In this problem, we will implement a variety of sklearn estimators.

## Part a)
Implement a transformer which standardizes a dataset according to the following formula

$$ X = \frac{X - \mu}{\sigma}$$ Where $\mu$ is the average and $\sigma$ is the standard deviation.

In [80]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
import pandas as pd
from sklearn.utils.validation import check_X_y, check_array
import numpy as np

class StandardizerTransformer(BaseEstimator, TransformerMixin):

    # applies tranformation to feature matrix 
    def transform(self, x, y=''):
        x = check_array(x)
        if self.n_features_in_ != len(x[0]):
            raise ValueError
        x = (x-self.mean_) / self.std_    
        
        return x # return the transformed x (new data)
    
    # calculate the "parameters" or the mean and standard deviation:
    def fit(self, x, y=''):
        x, y = check_X_y(x, y)
        self.n_features_in_ = len(x[0])
        self.mean_ = np.mean(x)
        self.std_ = np.std(x)
     
        return self

In [81]:
import pandas as pd
trial_df = [[1, 2, 3, 4, 2],
[1,2,3,4,2],
[0,0,0,0,0]]

y = [1,2,3]

trial_df


st = StandardizerTransformer()
model = st.fit(trial_df,y)
print(model.mean_)



1.6


In [82]:
from sklearn.utils.estimator_checks import check_estimator
check_estimator(StandardizerTransformer())


## Part b)

Implement a Regressor which predicts the average of the input values.  As we know this is not a very good model, we will set the poor score tag so that we can validate the model interface. 

Please note that just because this is not the greatest model, that does not mean it is not a useful one.  This model can provide a wonderful baseline with which to start.

In [83]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_is_fitted


class MeanModel(BaseEstimator, RegressorMixin):
    def _more_tags(self):
        return dict(poor_score=True)

    def fit(self, X, y=None):
        X, y = check_X_y(X, y)
        self.is_fitted_ = True
        self.n_features_in_ = len(X[0])
        self.mean_ = np.mean(X)
        return self

    def predict(self, X):
        X = check_array(X)
        check_is_fitted(self)
        return np.full(shape=X.shape[0], fill_value=self.mean_)
    

In [84]:
check_estimator(MeanModel())


## Problem 2

In this problem, we will explore some of the higher level constructs in sklearn.  Lets start by downloading the dataset from

https://archive.ics.uci.edu/dataset/2/adult

which is known as the census income dataset (see `adult.names` after the downloading of the data for more info).  We can do this with the following python function.



In [85]:
import requests
from pathlib import Path
URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/{}"
def download(url, filepath):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with filepath.open('wb') as fp:
            for chunk in r.iter_content():
                fp.write(chunk)
        
def download_adult_data(ignore_cache=False):
    data_path = Path('data')
    data_path.mkdir(exist_ok=True)
    files = ['adult.data', 'adult.names', 'adult.test']
    for file_ in files:
        filepath = data_path.joinpath(file_)
        if not ignore_cache and filepath.is_file():
            continue
        download(URL.format(file_), filepath)
    return [data_path.joinpath(f) for f in files]
download_adult_data()

[PosixPath('data/adult.data'),
 PosixPath('data/adult.names'),
 PosixPath('data/adult.test')]

Now we will load the data into appropriate datastructure  for processing with scikit-learn.  We use use `pandas` as a tool to import data.  Our goal is not to rewrite useful libraries but to use the tools available in order to do some interesting data stuff.

In [86]:
from dataclasses import dataclass
import numpy as np

@dataclass
class Dataset:
    X_train: np.array
    y_train: np.array
    X_test: np.array
    y_test: np.array

In [87]:
import numpy as np
import pandas as pd
def load_adult_data():
    download_adult_data()
    data = pd.read_csv(Path('data').joinpath('adult.data'), header=None).values
    test_data = pd.read_csv(Path('data').joinpath('adult.test'), header=None, skiprows=1).values
    return Dataset(
        X_train=data[:, :-1],
        y_train=data[:, -1],
        X_test=test_data[:, :-1],
        y_test=np.array([i.rstrip('.') for i in test_data[:, -1]])
    )


Now lets do some exploratory data analysis on this dataset. In particular, look at the following columns 

- age
- workclass
- education


And answer the following questions (with supporting evidence):

- Is there anything about this feature which may need to be "engineered" in order to make it more useful?
- Do you think that this feature has any predictive power?  Does this make intuitive sense to you?

In [88]:

dataset = load_adult_data()
# age = 0, workclass = 1, education = 3. 
for i in range(10):
    print("age =", dataset.X_train[i][0], 
          "workclass =", dataset.X_train[i][1], 
          "education =", dataset.X_train[i][3])
    
print("Min Age:", np.min(dataset.X_train[:,0]))
print("Max Age:", np.max(dataset.X_train[:,0]))

print("\nTypes of work:", np.unique(dataset.X_train[:,1]))

print("\nTypes of education:", np.unique(dataset.X_train[:,3]))



age = 39 workclass =  State-gov education =  Bachelors
age = 50 workclass =  Self-emp-not-inc education =  Bachelors
age = 38 workclass =  Private education =  HS-grad
age = 53 workclass =  Private education =  11th
age = 28 workclass =  Private education =  Bachelors
age = 37 workclass =  Private education =  Masters
age = 49 workclass =  Private education =  9th
age = 52 workclass =  Self-emp-not-inc education =  HS-grad
age = 31 workclass =  Private education =  Masters
age = 42 workclass =  Private education =  Bachelors
Min Age: 17
Max Age: 90

Types of work: [' ?' ' Federal-gov' ' Local-gov' ' Never-worked' ' Private'
 ' Self-emp-inc' ' Self-emp-not-inc' ' State-gov' ' Without-pay']

Types of education: [' 10th' ' 11th' ' 12th' ' 1st-4th' ' 5th-6th' ' 7th-8th' ' 9th'
 ' Assoc-acdm' ' Assoc-voc' ' Bachelors' ' Doctorate' ' HS-grad'
 ' Masters' ' Preschool' ' Prof-school' ' Some-college']


Is there anything about this feature which may need to be "engineered" in order to make it more useful?

Yes, finding the min and max of each feature, how many times each instance occurs, average, etc

Do you think that this feature has any predictive power? Does this make intuitive sense to you?

It can. There may be correlations with age and education level or even workclass and education. 

# Problem 3

Now we will start to do some basic feature engineering and start building up a set of models.

## Part a)

First we will start with the education feature.  While education is a categorical variable in the dataset, we do not expect all categories to be completely orthognal.  Lets create a transformer which will take the education categories and collapse all education levels below HS-grad to a single value "No-Degree".  And yes, there is an education-number feature as well, however, we want to do some exercises to learn :)

**NOTE**: Although coding standards and checking for correctness of transformations is important, we will be concerned here with only your output.  However, we will be testing edge cases, so make sure you are considering those.

In [89]:
from sklearn.base import BaseEstimator
class EducationTransformer(BaseEstimator, TransformerMixin):
  

    def transform(self, x, y=''):

        li = ['Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th', '11th', '12th']
                
        for i in range(len(x)):
            if x[i][0].strip() in li:
                x[i][0] = "No-Degree"
                    
        return x
        
    def fit(self, x, y=''):
        self.is_fitted_ = True
        self.n_features_in_ = x.shape[0]
        
        return self
 

In [90]:
d = load_adult_data()

transformed = EducationTransformer().fit_transform(d.X_train[:, 3:4])[:, 0]

In [91]:
d = load_adult_data()

transformed = EducationTransformer().fit_transform(d.X_train[:, 3:4])[:, 0]
assert 'No-Degree' in set(transformed)
assert '11th' not in set(transformed)


In [92]:
#test that it works
for i in range(100):
    print(d.X_train[i][3])

 Bachelors
 Bachelors
 HS-grad
No-Degree
 Bachelors
 Masters
No-Degree
 HS-grad
 Masters
 Bachelors
 Some-college
 Bachelors
 Bachelors
 Assoc-acdm
 Assoc-voc
No-Degree
 HS-grad
 HS-grad
No-Degree
 Masters
 Doctorate
 HS-grad
No-Degree
No-Degree
 HS-grad
 Bachelors
 HS-grad
 Some-college
 HS-grad
 HS-grad
 Assoc-acdm
 Some-college
 Bachelors
 Some-college
 Some-college
No-Degree
 Some-college
 HS-grad
 Some-college
 Assoc-acdm
No-Degree
 Bachelors
 Bachelors
 HS-grad
 HS-grad
 Bachelors
 HS-grad
 Masters
 Assoc-voc
 Assoc-voc
 Some-college
 HS-grad
 Prof-school
 Bachelors
 HS-grad
 Some-college
No-Degree
 Assoc-voc
 HS-grad
 HS-grad
 Bachelors
No-Degree
 HS-grad
 Doctorate
 Some-college
 HS-grad
 Some-college
 HS-grad
 Some-college
 Some-college
 Some-college
 Bachelors
 Bachelors
 Some-college
 Some-college
 HS-grad
 Assoc-acdm
No-Degree
No-Degree
No-Degree
 HS-grad
 Bachelors
 HS-grad
 HS-grad
 HS-grad
 HS-grad
 HS-grad
 Masters
No-Degree
 Doctorate
 Assoc-voc
 Some-college
 Some-col

## Part b)

Now its often the case that models will not handle features which are not numerical, we will learn more about this later in class, however, for this specific example, we can get an intuition by thinking about these education levels.  Many algorithms (for example linear regression) will require that we use numerical features.  However, if we use numerical features, we are making some implicit assumptions about the metric on this data, specifically the distance between values.  Lets take a simple example with the following data

|shirt_color|
|---|
|red|
|red|
|blue|
|green|

In this case, we could numerically encode this as 


|shirt_color|
|---|
|1|
|1|
|2|
|3|

and use a regression on the data, however, this would imply that red is somehow closer than to blue than to green.  In other words, our assignment of labels will affect the output.  Instead, we can reshape the matrix like so

|shirt_color_red|shirt_color_blue|shirt_color_green|
|---|---|---|
|1|0|0|
|1|0|0|
|0|1|0|
|0|0|1|

which removes the encoding issues.

Use the `OneHotcoder` from the sklearn library to build a more complex transformer which uses your previous transformer and then applies the one hot encoding on top of it.

There are two possible ways to do this, one is to use a higher order constructor like a `Pipeline` and the other is to use composition.  You can do either in principle but please use a `Pipeline` to do it here.


In [93]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

education_pipe = Pipeline([
    ("education", EducationTransformer()), 
    
    ("one hot coder", OneHotEncoder(handle_unknown='ignore'))
    
])

#education_pipe.fit(d.X_train)

In [94]:
d = load_adult_data()

transformed = education_pipe.fit_transform(d.X_train[:, 3:4])

assert transformed.shape[0] == d.X_train.shape[0]


In [95]:
#education_pipe.fit(d.X_train)

for i in range(10):
    print(i, "=", d.X_train[i])

0 = [39 ' State-gov' 77516 ' Bachelors' 13 ' Never-married' ' Adm-clerical'
 ' Not-in-family' ' White' ' Male' 2174 0 40 ' United-States']
1 = [50 ' Self-emp-not-inc' 83311 ' Bachelors' 13 ' Married-civ-spouse'
 ' Exec-managerial' ' Husband' ' White' ' Male' 0 0 13 ' United-States']
2 = [38 ' Private' 215646 ' HS-grad' 9 ' Divorced' ' Handlers-cleaners'
 ' Not-in-family' ' White' ' Male' 0 0 40 ' United-States']
3 = [53 ' Private' 234721 'No-Degree' 7 ' Married-civ-spouse'
 ' Handlers-cleaners' ' Husband' ' Black' ' Male' 0 0 40 ' United-States']
4 = [28 ' Private' 338409 ' Bachelors' 13 ' Married-civ-spouse'
 ' Prof-specialty' ' Wife' ' Black' ' Female' 0 0 40 ' Cuba']
5 = [37 ' Private' 284582 ' Masters' 14 ' Married-civ-spouse'
 ' Exec-managerial' ' Wife' ' White' ' Female' 0 0 40 ' United-States']
6 = [49 ' Private' 160187 'No-Degree' 5 ' Married-spouse-absent'
 ' Other-service' ' Not-in-family' ' Black' ' Female' 0 0 16 ' Jamaica']
7 = [52 ' Self-emp-not-inc' 209642 ' HS-grad' 9 '

## Part c)

Now we can apply our transformer to the feature matrix by making use of the scikit-learn object `ColumnTransformer`.  Use this to build a transformer which operates directly on the input feature matrix. Additionally, in order to make the auto grading work, please ensure the following:

- Please name your solution transformer `ct_education`.  
- Please make sure to pass through the rest of the columns. (check the `passthrough` options in docs)

In [96]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct_education = ColumnTransformer(
    [ ('EduOneHot', education_pipe, [3])],
    remainder = 'passthrough' )


In [97]:
res = ct_education.fit_transform(d.X_train)

assert res.shape[0] == d.X_train.shape[0]


## Part d)

Now fit a model using the following features:

- your transformed education column
- age
- hours-per-week
- capital-gain

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(d.X_train[:, [0, 3, 10, 12]], d.y_train)


# your transformed education column
# age : col 0
# hours-per-week: col 12
# capital-gain col 10



ValueError: could not convert string to float: ' Bachelors'